<a href="https://colab.research.google.com/github/Baishaki-sfdc/RAG-llamaindex-groq/blob/main/Chatwithdoc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install Packages

In [1]:
%pip install llama-index==0.10.18 llama-index-llms-groq==0.1.3 groq==0.4.2 llama-index-embeddings-huggingface==0.2.0

INFO: pip is looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of llama-parse to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-parse to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 79.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 19.4 MB/

storage index:store all the index of the documents to disk, to avoid reindexing ,
service index:python module that llamaindex provides,

### Import Libraries

In [2]:
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    ServiceContext,
    load_index_from_storage
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.groq import Groq
# import os
# from dotenv import load_dotenv
# load_dotenv()
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /usr/local/lib/python3.11/dist-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [3]:
#from google.colab import userdata
#GROQ_API_KEY = userdata.get('groq')

# GROQ_API_KEY = os.getenv("GROQ_API_KEY")
import os
os.environ["GROQ_API_KEY"] = "Your Groq Api key"
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

### Data Ingestion

In [4]:
# data ingestion
from llama_index.core import SimpleDirectoryReader
reader = SimpleDirectoryReader(input_files=["./healthyheart.pdf"])
documents = reader.load_data()

https://docs.llamaindex.ai/en/stable/module_guides/loading/simpledirectoryreader/

In [5]:
len(documents)

95

In [6]:
documents[4].metadata

{'page_label': '5',
 'file_name': 'healthyheart.pdf',
 'file_path': 'healthyheart.pdf',
 'file_type': 'application/pdf',
 'file_size': 2292377,
 'creation_date': '2025-02-19',
 'last_modified_date': '2025-02-19'}

In [7]:
documents[5].metadata

{'page_label': '6',
 'file_name': 'healthyheart.pdf',
 'file_path': 'healthyheart.pdf',
 'file_type': 'application/pdf',
 'file_size': 2292377,
 'creation_date': '2025-02-19',
 'last_modified_date': '2025-02-19'}

### Chunking

In [9]:
text_splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=200)
nodes = text_splitter.get_nodes_from_documents(documents, show_progress=True)

Parsing nodes:   0%|          | 0/95 [00:00<?, ?it/s]

https://docs.llamaindex.ai/en/stable/module_guides/loading/node_parsers/modules/

In [10]:
len(nodes)

100

In [11]:
nodes[0].metadata

{'page_label': '1',
 'file_name': 'healthyheart.pdf',
 'file_path': 'healthyheart.pdf',
 'file_type': 'application/pdf',
 'file_size': 2292377,
 'creation_date': '2025-02-19',
 'last_modified_date': '2025-02-19'}

https://chunkviz.up.railway.app/

### Embedding Model

In [12]:
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

https://huggingface.co/spaces/mteb/leaderboard

### Define LLM Model

In [13]:
llm = Groq(model="llama3-70b-8192", api_key=GROQ_API_KEY)

https://console.groq.com/docs/models

https://console.groq.com/keys

### Configure Service Context

In [14]:
service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=llm)

### Create Vector Store Index

In [15]:
vector_index = VectorStoreIndex.from_documents(documents, show_progress=True, service_context=service_context, node_parser=nodes)

Parsing nodes:   0%|          | 0/95 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/100 [00:00<?, ?it/s]

https://docs.llamaindex.ai/en/stable/module_guides/indexing/vector_store_index/

#### Persist/Save Index

In [16]:
vector_index.storage_context.persist(persist_dir="./storage_mini")

#### Define Storage Context

In [17]:
storage_context = StorageContext.from_defaults(persist_dir="./storage_mini")

https://docs.llamaindex.ai/en/stable/api_reference/storage/storage_context/

#### Load Index

In [18]:
index = load_index_from_storage(storage_context, service_context=service_context)

### Define Query Engine

In [19]:
query_engine = index.as_query_engine(service_context=service_context)

https://docs.llamaindex.ai/en/stable/module_guides/deploying/query_engine/

#### Feed in user query

https://docs.llamaindex.ai/en/stable/examples/prompts/prompts_rag/#viewingcustomizing-prompts

In [20]:
query = "how cholestrol affects heart?"
resp = query_engine.query(query)

In [21]:
print(resp.response)

High cholesterol puts you at risk for heart disease.


https://itsjb13.medium.com/building-a-rag-chatbot-using-llamaindex-groq-with-llama3-chainlit-b1709f770f55

https://docs.llamaindex.ai/en/stable/optimizing/production_rag/